In [1]:
import pandas as pd

# 통합된 데이터를 활동을 시작한 주를 1로 보고 시계열 정렬한다.

In [2]:
tr_label = pd.read_csv('../../train/train_label.csv')
tr = pd.read_csv('../3.파생변수/train_create.csv')
te = pd.read_csv('../3.파생변수/test_create.csv')
to = pd.concat([tr,te], sort=False).sort_values(by=['acc_id', 'wk']).reset_index(drop=True).fillna(0)

In [3]:
count_wk = to.groupby('acc_id').agg({'wk': 'count'}).reset_index().rename(columns={'wk': 'activated_wk'})
start = []
for i in count_wk['activated_wk']:
    start.extend(list(range(1, i+1)))
to['start_wk'] = start
to.drop('wk', axis=1, inplace=True)

In [4]:
non_week = ['acc_id',
            'total_guild_mem', 'guild_count', 'max_guild_mem', 'mean_guild_mem',
            'trade_acc_count', 'total_trade_count',
            'dup_rec_acc_count', 'dup_rec_count', 'dup_give_acc_count', 'dup_give_count']

wk = {}
for i in range(1, 9):
    wk[i] = to[to['start_wk']==i].reset_index(drop=True).drop('start_wk', axis=1)
    wk[i].rename(columns={j: j+'_{}'.format(i) for j in wk[i].columns if j not in non_week}, inplace=True)

In [5]:
spread = wk[1]
for i in range(2, 9):
    spread = pd.merge(spread, wk[i], how='outer')

spread = pd.merge(spread, count_wk, how='outer').fillna(0)
spread = pd.merge(spread, tr_label, how='outer').sort_values(by='acc_id').reset_index(drop=True)
spread.columns

Index(['acc_id', 'cnt_dt_1', 'play_time_1', 'npc_exp_1', 'npc_hongmun_1',
       'quest_exp_1', 'quest_hongmun_1', 'item_hongmun_1',
       'game_combat_time_1', 'get_money_1',
       ...
       'X27_8', 'X28_8', 'X29_8', 'X30_8', 'X31_8', 'X32_8', 'X33_8', 'X34_8',
       'activated_wk', 'label'],
      dtype='object', length=837)

In [6]:
spread.head(10)

,acc_id,cnt_dt_1,play_time_1,npc_exp_1,npc_hongmun_1,quest_exp_1,quest_hongmun_1,item_hongmun_1,game_combat_time_1,get_money_1,...,X27_8,X28_8,X29_8,X30_8,X31_8,X32_8,X33_8,X34_8,activated_wk,label
0,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,5,5671.650000,173339,31434,10267620,16640780,0,1426.383333,3.7493,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,2,week
1,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.200000,0,0,0,0,0,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,8,retained
2,00028f0ad71c07f48aef465fd0c98ba6e3f0f3de3d2f7d...,2,20.333333,28,0,8610,0,0,2.100000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,5,retained
3,0002a56a036206aea3a6c6ebc985df4a2d1987b562e7f0...,1,4.133333,0,0,0,4500000,0,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,7,NaN
4,00036e5b6a197c196fa10fc0ad4e2853b22294dff64d27...,3,486.716667,9758,23515,2588780,8848160,80000,161.733333,0.6909,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,6,month
5,00037fe8e712041a476b8b1b827583cbc34895844057f0...,1,3.283333,0,0,0,0,0,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,2,2month
6,0003b127aa1c0c34490db8817574482464aad9f99cffde...,7,356.766667,0,50640,0,198100,200000,159.150000,1.4074,...,-0.900407,0.916145,-1.204776,1.204776,0.0,0.0,0.108538,0.0,8,month
7,0003bfdebe2b5d46217e8fa5ab0f33cf186fb558435f20...,1,2.533333,0,0,0,0,0,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,5,NaN
8,0004733c4175d61e67d1ec9d3602f6c3341180800bc8bf...,5,4.766667,0,0,0,0,0,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,1,month
9,00047f6584e6bcfed540e1bc53651c0c27f02c5bc53091...,2,20.300000,28,0,8610,0,0,1.650000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,3,2month


In [7]:
new_tr = spread[~pd.isna(spread['label'])]
new_te = spread[pd.isna(spread['label'])].drop('label', axis=1)

In [8]:
new_tr.to_csv('train_start_spread.csv', index=False)
new_te.to_csv('test_start_spread.csv', index=False)